In [1]:
!pip install datasets pinecone --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Create Pinecone DBs for Evaluation

In [2]:
#Import Necessary Libraries

import numpy as np
import pandas as pd
import ast
import torch
from tqdm import tqdm
import pinecone
import multiprocessing as mp
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection, CLIPTextModelWithProjection
from PIL import Image

In [3]:
#Load Dataset from Aswath Parquet Files

import requests



#Identify Blank and Non-Blank Images
img = Image.new('RGB', (200, 200), color='white')
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
model.eval()
with torch.no_grad():
  image_inputs = processor(images=img, return_tensors="pt")
  # img_tensor = model.get_image_features(**image_inputs)
  img_tensor = model(**image_inputs).image_embeds.squeeze(0)
  white_img_array = img_tensor.detach().numpy()

def white_non_white(x, white_img_array):
  if (np.round(x, decimals = 2) == np.round(white_img_array, decimals = 2)).all():
    return 'Blank Image'
  else:
    return 'Valid Image'


#Create a List of Dataframes
df_list = []

for i in tqdm(range(3)):
  df_name = '/content/Pet_Supplies_tokenized_results_' + str(i+1) + '.parquet'
  df = pd.read_parquet(df_name)
  df['text_embeddings'] = df['tokenized_text']
  df['image_embeddings'] = df['tokenized_image']

  df = df.loc[df['similarity'] > 0.2]
  df = df.reset_index(drop = True)

  for j in range(len(df)):
    # df.loc[j, 'image_type'] = white_non_white(df.loc[i, 'image_embeddings'], white_img_array, df.loc[i, 'image_url'])
    df.loc[j, 'image_type'] = white_non_white(df.loc[i, 'image_embeddings'], white_img_array)

  df_list.append(df)
  print(f"Completed Dataframe {df_name}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

Completed Dataframe /content/Pet_Supplies_tokenized_results_1.parquet


 67%|██████▋   | 2/3 [00:53<00:26, 26.81s/it]

Completed Dataframe /content/Pet_Supplies_tokenized_results_2.parquet


100%|██████████| 3/3 [01:08<00:00, 22.68s/it]

Completed Dataframe /content/Pet_Supplies_tokenized_results_3.parquet


In [4]:
#Keep only products that do not have white images
for df in tqdm(df_list):
  df = df.loc[df['image_type'] == 'Valid Image']
  df = df.reset_index(drop = True)

100%|██████████| 3/3 [00:00<00:00, 14.31it/s]


In [5]:
#Get count of total number of Observations remaining
length = 0

for i in df_list:
  length += len(i)

length

252675

In [6]:
final_df = pd.concat(df_list)
final_df = final_df.reset_index(drop = True)
final_df

,parent_asin,title,description,image_url,tokenized_text,tokenized_image,similarity,text_embeddings,image_embeddings,image_type
0,B00XJG2SLG,Hurtta Pet Collection 14-Inch Padded Y-Harness...,Hurtta harnesses are suitable for active walks...,https://m.media-amazon.com/images/I/51t7obbDd+...,"[0.19841735, -0.12057829, -0.25951636, 0.44316...","[-0.24335627, 0.07835907, -0.26544097, 0.46217...",0.348417,"[0.19841735, -0.12057829, -0.25951636, 0.44316...","[-0.24335627, 0.07835907, -0.26544097, 0.46217...",Valid Image
1,B01MQTWB5H,4 Pack - 4 Inch Ring Filter Socks 200 Micron -...,Micron filter bags provide excellent mechanica...,https://m.media-amazon.com/images/I/31T4ekKFOr...,"[0.20698957, -0.063747704, -0.13466147, -0.042...","[0.098101966, 0.57478565, -0.17086814, -0.2226...",0.311142,"[0.20698957, -0.063747704, -0.13466147, -0.042...","[0.098101966, 0.57478565, -0.17086814, -0.2226...",Valid Image
2,B00F3JRLYQ,Wysong Optimal Adult Canine Formula Dry Dog Fo...,For the past 35 years the Wysong goal and comm...,https://m.media-amazon.com/images/I/515T9QIcI-...,"[0.2611456, -0.110724404, -0.42732027, -0.0265...","[-0.17931566, 0.08053039, -0.21038799, -0.1327...",0.408751,"[0.2611456, -0.110724404, -0.42732027, -0.0265...","[-0.17931566, 0.08053039, -0.21038799, -0.1327...",Valid Image
3,B0006L145S,"Vitakraft Guinea Pig Happy Frutti Treat, 6 Oun...",Vitakraft's Happy Frutti treats for Guinea Pig...,https://m.media-amazon.com/images/I/41IpNhxh-l...,"[0.51642853, -0.42676795, -0.005760301, 0.0429...","[0.63313496, -0.4973585, 0.5789876, -0.0006987...",0.410419,"[0.51642853, -0.42676795, -0.005760301, 0.0429...","[0.63313496, -0.4973585, 0.5789876, -0.0006987...",Valid Image
4,B074SVVSK7,K9PROLINE [Waterproof] Professional Dog Leash ...,The Alpha Series is our flagship line of profe...,https://m.media-amazon.com/images/I/41Fh-yQq+Z...,"[0.06483249, -0.1931954, -0.44384804, -0.04827...","[-0.10636324, -0.1393775, -0.19780509, 0.16438...",0.292474,"[0.06483249, -0.1931954, -0.44384804, -0.04827...","[-0.10636324, -0.1393775, -0.19780509, 0.16438...",Valid Image
...,...,...,...,...,...,...,...,...,...,...
252670,B07Z1R1WF2,balacoo Natural Wood Stand Platfrom Wood Playg...,"Description, Not only human need toys, but als...",https://m.media-amazon.com/images/I/31Zr0gQf8q...,"[0.33570555, -0.33770478, -0.37514743, 0.13518...","[0.16728304, -0.070958056, 0.16254357, 0.04094...",0.384179,"[0.33570555, -0.33770478, -0.37514743, 0.13518...","[0.16728304, -0.070958056, 0.16254357, 0.04094...",Valid Image
252671,B08DFQMJW6,"LanYao Cat Dog Bubble Carrier Backpack, Pet Ca...","Product Features:, Durable,comfortable and lig...",https://m.media-amazon.com/images/I/41miLad0Gd...,"[0.07208563, -0.43625358, -0.06303226, -0.0385...","[-0.04477954, 0.022132456, -0.084290095, -0.05...",0.366302,"[0.07208563, -0.43625358, -0.06303226, -0.0385...","[-0.04477954, 0.022132456, -0.084290095, -0.05...",Valid Image
252672,B00H4TVEVY,uxcell Silicone Aquarium Air Oxygen Circulate ...,Suitable to be used for Oxygen supply in all k...,https://m.media-amazon.com/images/I/31nlqP9o5a...,"[0.14079247, -0.34255862, -0.3710283, 0.089686...","[0.087914936, -0.06368561, -0.23950747, 0.0297...",0.308451,"[0.14079247, -0.34255862, -0.3710283, 0.089686...","[0.087914936, -0.06368561, -0.23950747, 0.0297...",Valid Image
252673,B01LX50GW8,MAJORDOG Frog with Plush Dog Retrieval Toy,The MAJORDOG frog with plush is a great motiva...,https://m.media-amazon.com/images/I/414D2qYAMM...,"[0.43867174, -0.2083219, -0.30451193, 0.088778...","[0.42116612, 0.039335698, -0.10396848, 0.21891...",0.348079,"[0.43867174, -0.2083219, -0.30451193, 0.088778...","[0.42116612, 0.039335698, -0.10396848, 0.21891...",Valid Image


In [7]:
#Load Filtered Dataset and keep only those products that belong to selected 'main_category' values

def filter_main_category(main_category_list):
  filtered_df = pd.read_parquet('/content/filtered_dataset_pet_supplies.parquet')
  filtered_df = filtered_df.loc[filtered_df['main_category'].isin(main_category_list)]
  filtered_df = filtered_df.reset_index(drop = True)
  return filtered_df


filtered_df = filter_main_category(['Toys & Games', 'Health & Personal Care'])
filtered_df = filtered_df[['parent_asin', 'main_category']]
filtered_df

,parent_asin,main_category
0,B0B613T3J4,Toys & Games
1,B01J9R155I,Toys & Games
2,B0B71LCXFL,Toys & Games
3,B06Y67P1C1,Health & Personal Care
4,B07RVCVL79,Health & Personal Care
...,...,...
2115,B0C3CP5VBB,Health & Personal Care
2116,B08X1BJD33,Toys & Games
2117,B08FFJ79Z5,Health & Personal Care
2118,B08ZFVZPHM,Health & Personal Care


In [8]:
#Perform Intersection on Parent ASIN between 'final_df' and 'filtered_df'
final_df = final_df.merge(filtered_df, on = 'parent_asin', how = 'inner')
final_df

,parent_asin,title,description,image_url,tokenized_text,tokenized_image,similarity,text_embeddings,image_embeddings,image_type,main_category
0,B0B613T3J4,Sea Monkeys 80483 Volcano Zoo,toys & games,https://m.media-amazon.com/images/I/51kNLFn7ML...,"[0.32822427, -0.25419366, -0.12156683, 0.01994...","[0.50233555, -0.07785687, 0.015347764, 0.01359...",0.370176,"[0.32822427, -0.25419366, -0.12156683, 0.01994...","[0.50233555, -0.07785687, 0.015347764, 0.01359...",Valid Image,Toys & Games
1,B01J9R155I,"Rubie's Sushi Pet Costume, X-Large",Sushi pet costume includes soft plush nigiri j...,https://m.media-amazon.com/images/I/412IjUlIWb...,"[0.61379653, 0.10788745, -0.23695296, 0.050052...","[0.9873979, -0.39027596, 0.09543973, -0.038132...",0.381395,"[0.61379653, 0.10788745, -0.23695296, 0.050052...","[0.9873979, -0.39027596, 0.09543973, -0.038132...",Valid Image,Toys & Games
2,B0B71LCXFL,12 Packs Dog Talking Button Set for Communicat...,Features: Recordable answer buttons can not on...,https://m.media-amazon.com/images/I/51yzo7hWcE...,"[0.28109077, -0.19451754, 0.018790767, 0.04688...","[0.34481174, -0.26435956, -0.102661654, 0.1490...",0.386195,"[0.28109077, -0.19451754, 0.018790767, 0.04688...","[0.34481174, -0.26435956, -0.102661654, 0.1490...",Valid Image,Toys & Games
3,B06Y67P1C1,Brown Sugar Pet Store 7 Piece Sugar Glider Red...,"Package Include:, 1 x Velvet Red Devil Dome Be...",https://m.media-amazon.com/images/I/51J5RJlYkr...,"[0.44446385, 0.17957667, -0.12516074, 0.173476...","[0.31771135, 0.26470298, -0.3632985, 0.1684223...",0.329143,"[0.44446385, 0.17957667, -0.12516074, 0.173476...","[0.31771135, 0.26470298, -0.3632985, 0.1684223...",Valid Image,Health & Personal Care
4,B07RVCVL79,"Brine Shrimp Direct Spirulina Powder, 8 oz",8 oz Spirulina Powder. Spirulina is a blue-gre...,https://m.media-amazon.com/images/I/41m-KDthEk...,"[0.035320215, -0.5079438, -0.28358084, -0.0186...","[-0.11981461, 0.22478442, -0.16519496, -0.0195...",0.384606,"[0.035320215, -0.5079438, -0.28358084, -0.0186...","[-0.11981461, 0.22478442, -0.16519496, -0.0195...",Valid Image,Health & Personal Care
...,...,...,...,...,...,...,...,...,...,...,...
2115,B0C3CP5VBB,"WABRINY Dog Use Liquid,Safe and Effective (0.5...",Easy to use.,https://m.media-amazon.com/images/I/51iKBQVHrJ...,"[-0.31049865, -0.12140659, 0.044756975, -0.131...","[-0.22809379, 0.18459095, 0.02016341, -0.22722...",0.293099,"[-0.31049865, -0.12140659, 0.044756975, -0.131...","[-0.22809379, 0.18459095, 0.02016341, -0.22722...",Valid Image,Health & Personal Care
2116,B08X1BJD33,English Bulldog Plush Puppy with Harness and E...,Want to take your favorite four legged friend ...,https://m.media-amazon.com/images/I/41SqITBXpQ...,"[0.11424414, -0.14652523, -0.4091345, -0.13208...","[0.3303455, -0.31651348, -0.070588574, -0.1109...",0.361958,"[0.11424414, -0.14652523, -0.4091345, -0.13208...","[0.3303455, -0.31651348, -0.070588574, -0.1109...",Valid Image,Toys & Games
2117,B08FFJ79Z5,Castle Rid-It Odor Eliminator Room Air Freshen...,"CASTLE RID IT ODOR ELIMINATOR, PUT AN END TO P...",https://m.media-amazon.com/images/I/51BvKGrHvm...,"[0.019041762, -0.316109, -0.1962876, -0.029139...","[-0.39335325, 0.37702233, 0.2800838, -0.324733...",0.393118,"[0.019041762, -0.316109, -0.1962876, -0.029139...","[-0.39335325, 0.37702233, 0.2800838, -0.324733...",Valid Image,Health & Personal Care
2118,B08ZFVZPHM,Adventure Dog Medical Kit - Vet in a Box,Playing in the backyard or hiking the nearby t...,https://m.media-amazon.com/images/I/518nJNE+o1...,"[-0.037170663, -0.19597447, -0.4471586, 0.0002...","[-0.18035957, 0.033047736, -0.09321335, 0.1475...",0.397752,"[-0.037170663, -0.19597447, -0.4471586, 0.0002...","[-0.18035957, 0.033047736, -0.09321335, 0.1475...",Valid Image,Health & Personal Care


In [9]:
from google.colab import userdata

pinecone_api = "42b77faa-1fbe-48e1-a625-1870dee3e5fd"

pc = pinecone.Pinecone(api_key=pinecone_api)
index = pc.Index("pet-supplies-eval-image",
                 host = "https://pet-supplies-eval-image-d1g0fdz.svc.aped-4627-b74a.pinecone.io")


In [10]:
X = final_df[['parent_asin', 'title', 'description', 'image_url', 'similarity', 'main_category', 'text_embeddings', 'image_embeddings']]
X.rename(columns = {'similarity': 'text_image_cosine_similarity'}, inplace = True)
X['add_embeddings'] = X['text_embeddings'] + X['image_embeddings']
X.drop(columns = ['text_embeddings'], inplace = True)
X.to_csv('Eval_Dataset_With_Embeddings.csv')
X.to_parquet('Eval_Dataset_With_Embeddings.parquet')

<ipython-input-10-0717ac00311e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(columns = {'similarity': 'text_image_cosine_similarity'}, inplace = True)
<ipython-input-10-0717ac00311e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['add_embeddings'] = X['text_embeddings'] + X['image_embeddings']
<ipython-input-10-0717ac00311e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [11]:
X

,parent_asin,title,description,image_url,text_image_cosine_similarity,main_category,image_embeddings,add_embeddings
0,B0B613T3J4,Sea Monkeys 80483 Volcano Zoo,toys & games,https://m.media-amazon.com/images/I/51kNLFn7ML...,0.370176,Toys & Games,"[0.50233555, -0.07785687, 0.015347764, 0.01359...","[0.83055985, -0.33205053, -0.10621907, 0.03353..."
1,B01J9R155I,"Rubie's Sushi Pet Costume, X-Large",Sushi pet costume includes soft plush nigiri j...,https://m.media-amazon.com/images/I/412IjUlIWb...,0.381395,Toys & Games,"[0.9873979, -0.39027596, 0.09543973, -0.038132...","[1.6011944, -0.2823885, -0.14151323, 0.0119206..."
2,B0B71LCXFL,12 Packs Dog Talking Button Set for Communicat...,Features: Recordable answer buttons can not on...,https://m.media-amazon.com/images/I/51yzo7hWcE...,0.386195,Toys & Games,"[0.34481174, -0.26435956, -0.102661654, 0.1490...","[0.62590253, -0.4588771, -0.08387089, 0.195905..."
3,B06Y67P1C1,Brown Sugar Pet Store 7 Piece Sugar Glider Red...,"Package Include:, 1 x Velvet Red Devil Dome Be...",https://m.media-amazon.com/images/I/51J5RJlYkr...,0.329143,Health & Personal Care,"[0.31771135, 0.26470298, -0.3632985, 0.1684223...","[0.7621752, 0.44427964, -0.48845923, 0.3418986..."
4,B07RVCVL79,"Brine Shrimp Direct Spirulina Powder, 8 oz",8 oz Spirulina Powder. Spirulina is a blue-gre...,https://m.media-amazon.com/images/I/41m-KDthEk...,0.384606,Health & Personal Care,"[-0.11981461, 0.22478442, -0.16519496, -0.0195...","[-0.0844944, -0.28315938, -0.4487758, -0.03818..."
...,...,...,...,...,...,...,...,...
2115,B0C3CP5VBB,"WABRINY Dog Use Liquid,Safe and Effective (0.5...",Easy to use.,https://m.media-amazon.com/images/I/51iKBQVHrJ...,0.293099,Health & Personal Care,"[-0.22809379, 0.18459095, 0.02016341, -0.22722...","[-0.53859246, 0.06318436, 0.06492038, -0.35837..."
2116,B08X1BJD33,English Bulldog Plush Puppy with Harness and E...,Want to take your favorite four legged friend ...,https://m.media-amazon.com/images/I/41SqITBXpQ...,0.361958,Toys & Games,"[0.3303455, -0.31651348, -0.070588574, -0.1109...","[0.44458964, -0.4630387, -0.4797231, -0.243009..."
2117,B08FFJ79Z5,Castle Rid-It Odor Eliminator Room Air Freshen...,"CASTLE RID IT ODOR ELIMINATOR, PUT AN END TO P...",https://m.media-amazon.com/images/I/51BvKGrHvm...,0.393118,Health & Personal Care,"[-0.39335325, 0.37702233, 0.2800838, -0.324733...","[-0.3743115, 0.060913324, 0.0837962, -0.353872..."
2118,B08ZFVZPHM,Adventure Dog Medical Kit - Vet in a Box,Playing in the backyard or hiking the nearby t...,https://m.media-amazon.com/images/I/518nJNE+o1...,0.397752,Health & Personal Care,"[-0.18035957, 0.033047736, -0.09321335, 0.1475...","[-0.21753024, -0.16292673, -0.54037195, 0.1477..."


In [ ]:
#Using Multiple CPU Cores to upsertion to Pinecone
import sys
import json


# Function to handle upsertion for each chunk
def upload_chunk_to_pinecone(dataframe_chunk):
    # text_batch = []
    # image_batch = []
    batch = []

    for i, row in tqdm(dataframe_chunk.iterrows()):
        total_size = 0

        # Define unique vector ID (e.g., 'vector_0', 'vector_1', etc.)

        vector_id = row['parent_asin']

        # Extract vector (assuming it's a numpy array)
        # vector = (row['text_embeddings'] + row['image_embeddings'])/2 #Average
        # vector = row['text_embeddings'] + row['image_embeddings'] #Addition
        # vector = np.concatenate((row['text_embeddings'], row['image_embeddings']), axis = 0) #Concatenation
        vector = row['image_embeddings'] #Only Image


        # Create metadata dictionary
        metadata = {
            'parent_asin': row['parent_asin'],
            'title': row['title'],
            'description': row['description'],
            # 'features': row['features'],
            'image_url': row['image_url'],
            'text_image_cosine_similarity': row['similarity'],
            'main_category': row['main_category']
            # 'image_type': row['image_type']
        }


        max_size = 40960

        metadata_json = json.dumps(metadata)
        total_size += len(metadata_json.encode('utf-8'))

        #40960 bytes is the max metadata size that a vector can have on Pinecone
        #If size exceeds limit, reduce description length
        while total_size > 40960:
          new_length = int(len(metadata['description'])/2)
          metadata['description'] = row['description'][:new_length]
          metadata_json = json.dumps(metadata)
          total_size = len(metadata_json.encode('utf-8'))
        batch.append((vector_id, vector, metadata))

        # Upload batch in chunks of 500 vectors ---> Switch to 200 when using concat
        if len(batch) == 500:
          index.upsert(vectors=batch)
          batch = []

    # Upload any remaining vectors in the batch
    if batch:
        index.upsert(vectors=batch)


# Function to perform multiprocessing across all dataframes
def parallel_upload(dataframes, num_processes=2):
    with mp.Pool(num_processes) as pool:
        pool.map(upload_chunk_to_pinecone, dataframes)


df_list = [final_df]
parallel_upload(df_list, num_processes=4)


2120it [00:12, 167.00it/s]
